## PM Check-in Data Prep

In [1]:
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
import synapseclient
from synapseclient import Activity, Schema, Table, as_table_columns
import numpy as np

InteractiveShell.ast_node_interactivity = 'all'
syn = synapseclient.Synapse()
syn.login()


UPGRADE AVAILABLE

A more recent version of the Synapse Client (2.0.0) is available. Your version (1.9.4) can be upgraded by typing:
    pip install --upgrade synapseclient

Python Synapse Client version 2.0.0 release notes

https://python-docs.synapse.org/build/html/news.html



Welcome, Sean Deering!



### Get Raw PM-Checkin Data

In [2]:
pmcheckin_raw_id = 'syn7117927'
pmcheckin = pd.read_csv(syn.get(pmcheckin_raw_id).path,sep='\t',error_bad_lines=False)

Skipping line 18432: expected 18 fields, saw 22



### Replace white space with NaN

In [3]:
pmcheckin = pmcheckin.replace(r'^\s*$', np.nan, regex=True)

### Process Raw PM-Checkin data

In [4]:
#rename these variables for consistency
pmcheckin = pmcheckin.rename(columns = 
                             { 'PMCH-1':'PMCH1',
                              'PMCH-2a':'PMCH2A',
                              'PMCH-3':'PMCH3'})

In [5]:
#remove brackets
def remove_brackets(col):
    return( col.astype(str).str.replace(']', '').str.replace('[',''))
pmcheckin.NapCount=remove_brackets(pmcheckin.NapCount)
pmcheckin.PMCH1=remove_brackets(pmcheckin.PMCH1)
pmcheckin.PMCH3=remove_brackets(pmcheckin.PMCH3)

#recode variables for PMCH-1 into what they were supposed to be
pmcheckin['PMCH1'].replace('Very poor',1,inplace=True)
pmcheckin['PMCH1'].replace('Poor',2,inplace=True)
pmcheckin['PMCH1'].replace('Fair',3,inplace=True)

### Filter based on age

In [6]:
underage_participants = pd.read_csv( syn.get('syn21905452').path, sep="\t")
pmcheckin = pmcheckin[~pmcheckin.participantId.isin(underage_participants.participantId)]

### Remove test accounts

In [7]:
test_accounts = pd.read_excel(syn.get('syn21958537').path)

pmcheckin = pmcheckin[~pmcheckin.participantId.isin(test_accounts.participantId)]

### Clean Alcohol Related Columns (outlier code commented out, all data provided).

In [8]:
alcohol_related_cols = ['alcohol_only', 'alcohol&medication_alcohol', 'caffeine&alcohol_alcohol', 'caffeine&alcohol&medication_alcohol']
alcohol = pmcheckin[alcohol_related_cols]
alcohol = alcohol.sum(axis=1, skipna=True).astype(int)
alcohol.quantile([.1, .5, .6, .8, .85, .90, .95, .97, 0.99, 0.999])
alcohol.describe()

## If the #alcohol drinks is > .99 percentile of the sample in this case > 7 , we list that value is as
#more relaxed than normative data
#alcohol[ alcohol > 7 ] = 'LOW DATA QUALITY'
#alcohol.value_counts()

pmcheckin['alcohol'] = alcohol

0.100     0.0
0.500     0.0
0.600     0.0
0.800     0.0
0.850     1.0
0.900     2.0
0.950     3.0
0.970     4.0
0.990     7.0
0.999    24.0
dtype: float64

count    4.313200e+04
mean     2.862307e+05
std      5.944498e+07
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.234568e+10
dtype: float64

### Clean Caffeine Related Columns (outlier code commented out, all data provided).

In [9]:
caffeine_related_cols = ['caffeine_only','caffeine&medication_caffeine','caffeine&alcohol_caffeine','caffeine&alcohol&medication_caffeine' ]
caffeine = pmcheckin[caffeine_related_cols]

caffeine = caffeine.sum(axis=1, skipna=True).astype(int)
caffeine.quantile([.1, .5, .6, .8, .85, .90, .95, .97, 0.99, 0.999, 1])
caffeine.describe()

## If the #caffeine drinks is > .99 percentile of the sample in this case > 32 , we list that value is as 
#caffeine[ caffeine > 32 ] = 'LOW DATA QUALITY'
#caffeine.value_counts()

pmcheckin['caffeine'] = caffeine

0.100    0.000000e+00
0.500    2.000000e+00
0.600    2.000000e+00
0.800    4.000000e+00
0.850    6.000000e+00
0.900    1.000000e+01
0.950    1.600000e+01
0.970    2.400000e+01
0.990    3.200000e+01
0.999    6.400000e+01
1.000    3.001827e+10
dtype: float64

count    4.313200e+04
mean     6.959668e+05
std      1.445393e+08
min      0.000000e+00
25%      0.000000e+00
50%      2.000000e+00
75%      4.000000e+00
max      3.001827e+10
dtype: float64

### Drop Older Alcohol and Caffeine related cols 

In [10]:
alcohol_related_cols.extend(caffeine_related_cols)
removeCols = list(set(alcohol_related_cols))
pmcheckin = pmcheckin.drop(removeCols, axis=1)

### Clean PMCH2A (Nap Lengths) - (outlier code commented out, all data provided).

In [11]:
pmcheckin.PMCH2A.quantile([.01, .05, .5, .6, .8, .85, .90, .95, .97, 0.99, 0.999, 1])
pmcheckin.PMCH2A.describe()

0.010         1.00
0.050         2.00
0.500        30.00
0.600        45.00
0.800        90.00
0.850       120.00
0.900       120.00
0.950       180.00
0.970       219.92
0.990       300.00
0.999       600.00
1.000    207888.00
Name: PMCH2A, dtype: float64

count     11569.000000
mean         72.733771
std        1933.660850
min           0.000000
25%          15.000000
50%          30.000000
75%          65.000000
max      207888.000000
Name: PMCH2A, dtype: float64

In [12]:
## If the nap duration is > .99 percentile of the sample in this case > 300 (5 hours) , we list that value is as 
#pmcheckin.PMCH2A[ pmcheckin.PMCH2A > 300 ] = 'LOW DATA QUALITY'
#pmcheckin.PMCH2A.value_counts()

## Create External Copy of the data

In [13]:
#download the data 
sharing_info = pd.read_excel(syn.get('syn21958546').path)
healthCodes_with_broadsharing = sharing_info[sharing_info.sharing == 'all_qualified_researchers']

EXTERNAL_PMCHECKIN_DATA = pmcheckin[pmcheckin.participantId.isin(healthCodes_with_broadsharing['participant id'])]

#drop columns with free text fields
EXTERNAL_PMCHECKIN_DATA = EXTERNAL_PMCHECKIN_DATA.drop(columns=['medication_only','caffeine&medication_medication',
                                                                'alcohol&medication_medication',
                                                                'caffeine&alcohol&medication_medication'], axis=1)
EXTERNAL_PMCHECKIN_DATA.shape

(27380, 8)

### Number of unique participants

In [14]:
len(EXTERNAL_PMCHECKIN_DATA.participantId.unique())

4313

### Total number of observations

In [15]:
len(EXTERNAL_PMCHECKIN_DATA)

27380

### External - Upload to Synapse

In [16]:
SH_EXTERNAL_PROJECT = 'syn18492837'
table_schema_external = Schema(name='PM Check-in',
                               columns=as_table_columns(EXTERNAL_PMCHECKIN_DATA),
                               parent=SH_EXTERNAL_PROJECT)
pmCheckin_synTable_external = syn.store(Table(table_schema_external,EXTERNAL_PMCHECKIN_DATA))

### Internal - Upload to Synapse

In [17]:
SH_INTERNAL_PROJECT = 'syn7066726'
table_schema_internal = Schema(name='PM Check-in Internal',
                               columns=as_table_columns(pmcheckin),
                               parent=SH_INTERNAL_PROJECT)
pmCheckin_synTable_internal = syn.store(Table(table_schema_internal,pmcheckin))

In [18]:
activity=Activity(name= 'PM Check-in', 
                  description='Process and convert raw data to table format', 
                  used=pmcheckin_raw_id, 
                  executed='https://github.com/apratap/SleepHealth_Data_Release/blob/master/Create_PM_Checkin.ipynb')
#Internal 
syn.setProvenance(pmCheckin_synTable_internal, activity)

#External
syn.setProvenance(pmCheckin_synTable_external, activity)

{u'createdBy': u'3334346',
 u'createdOn': u'2020-07-20T18:58:54.786Z',
 u'description': u'Process and convert raw data to table format',
 u'etag': u'93b2073e-6f88-4c16-9c4e-ac409c5f5de5',
 u'id': u'10282150',
 u'modifiedBy': u'3334346',
 u'modifiedOn': u'2020-07-20T18:58:54.786Z',
 u'name': u'PM Check-in',
 u'used': [{u'concreteType': u'org.sagebionetworks.repo.model.provenance.UsedEntity',
   u'reference': {u'targetId': u'syn7117927', u'targetVersionNumber': 784},
   u'wasExecuted': False},
  {u'concreteType': u'org.sagebionetworks.repo.model.provenance.UsedURL',
   u'name': u'https://github.com/apratap/SleepHealth_Data_Release/blob/master/Create_PM_Checkin.ipynb',
   u'url': u'https://github.com/apratap/SleepHealth_Data_Release/blob/master/Create_PM_Checkin.ipynb',
   u'wasExecuted': True}]}

{u'createdBy': u'3334346',
 u'createdOn': u'2020-07-20T18:59:16.843Z',
 u'description': u'Process and convert raw data to table format',
 u'etag': u'be4557c8-41a3-4328-a876-18b05a28b8cd',
 u'id': u'10282151',
 u'modifiedBy': u'3334346',
 u'modifiedOn': u'2020-07-20T18:59:16.843Z',
 u'name': u'PM Check-in',
 u'used': [{u'concreteType': u'org.sagebionetworks.repo.model.provenance.UsedEntity',
   u'reference': {u'targetId': u'syn7117927', u'targetVersionNumber': 784},
   u'wasExecuted': False},
  {u'concreteType': u'org.sagebionetworks.repo.model.provenance.UsedURL',
   u'name': u'https://github.com/apratap/SleepHealth_Data_Release/blob/master/Create_PM_Checkin.ipynb',
   u'url': u'https://github.com/apratap/SleepHealth_Data_Release/blob/master/Create_PM_Checkin.ipynb',
   u'wasExecuted': True}]}